# Information Retrieval, Extraction and Integration
## Assignment 3: ML Ranking Assignment
### EIT Digital Innovation - Data Science 
- Davis Siemens
- Inés Simón del Collado
- Xiya Sun

In [1]:
import requests
import pandas as pd
import random

In [2]:
# Load Excel file
excel_file = "loinc_dataset-v2.xlsx"

# API Base URL (example, update as needed)
api_url = "https://loinc.regenstrief.org/searchapi/loincs"

# Authentication credentials
auth = ("davissiemens", "jejben-3rykVi-fejzaf")

In [3]:
# Function to get ranking for a LOINC code
def get_loinc_data(loinc_num):
    params = {"query": loinc_num, "rows": 1}  # Search parameter
    response = requests.get(api_url, params=params, auth=auth)

    if response.status_code == 200:
        data = response.json()
        if "Results" in data and len(data["Results"]) > 0:
            # Extract the COMMON_TEST_RANK
            return data["Results"][0].get("COMMON_TEST_RANK", "No Rank Found")
    return "Not Found"

In [4]:
# Function to check if a LOINC code is related to "glucose in blood"
def check_query(loinc_code, query_search):
    # Searching for "glucose in blood" in the LOINC database
    params = {"query": query_search, "rows": 800}  # Adjust query for glucose search
    response = requests.get(api_url, params=params, auth=auth)  # API call with authentication

    if response.status_code == 200:
        data = response.json()
        # Check if any of the results have the same LOINC code
        for result in data.get("Results", []):
            if result.get("LOINC_NUM") == loinc_code:
                return 1
    return 0

In [5]:
# Read all sheets from the Excel file
excel_sheets = pd.read_excel(excel_file, sheet_name=None, skiprows=2)

# Prepare a dictionary to hold the results
result_dict = {}

# Loop through each sheet in the Excel file
for sheet_name, df in excel_sheets.items():
    print(f"Processing sheet: {sheet_name}")
    # Assuming LOINC codes are in a column named 'LOINC Code'
    df["rank"] = df["loinc_num"].astype(str).apply(get_loinc_data)

    df["inSearch"] = df["loinc_num"].astype(str).apply(lambda x: check_query(x, sheet_name))

    # Save the results for each sheet in the result_dict
    result_dict[sheet_name] = df

# Save the results to a new Excel file with multiple sheets
with pd.ExcelWriter("./loinc_ranks_query.xlsx") as writer:
    for sheet_name, df in result_dict.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("LOINC rankings saved to loinc_ranks_query.xlsx")

Processing sheet: glucose in blood
Processing sheet: bilirubin in plasma
Processing sheet: White blood cells count
LOINC rankings saved to loinc_ranks_query.xlsx


## Adding more terms

In [3]:
# gets all related, and then we take xnumber
def get_allrelated_loincs(query, num_results=10):
    """
    Fetches relevant/unrelevant documents from the LOINC API based on search query (sheets names)
    """
    params = {"query": query, "rows": num_results*100}
    response = requests.get(api_url, params=params, auth=auth)

    results = []
    if response.status_code == 200:
        data = response.json()
        for result in data.get("Results", []):
            results.append({
                "loinc_num": result.get("LOINC_NUM", "Unknown"),
                "long_common_name": result.get("LONG_COMMON_NAME", "Unknown"),
                "component": result.get("COMPONENT", "Unknown"),
                "system": result.get("SYSTEM", "Unknown"),
                "property": result.get("PROPERTY", "Unknown"),
                "rank": result.get("COMMON_TEST_RANK", "No Rank Found"),
                "inSearch": 1  # Mark as related
            })
    return pd.DataFrame(results)

In [ ]:
# We create and storage a dataset from LOINC, so we dont have to run the analysis many times
# from this dataset, we will build the testing dataset and get the top unrelated.
def get_top_loinc_entries(min_rank=10000, num_results=5000):
    """
    Fetches LOINC entries where COMMON_TEST_RANK > min_rank and selects the top 'num_results'.
    """
    params = {
        "query": "*",
        "rows": num_results * 10,  # Fetch more to ensure we have enough ranked results
    }
    
    response = requests.get(api_url, params=params, auth=auth)
    
    if response.status_code != 200:
        print("Error fetching data from LOINC API")
        return pd.DataFrame()  # Return empty DataFrame on failure

    data = response.json()
    results = []

    for result in data.get("Results", []):
        rank = result.get("COMMON_TEST_RANK", None)
        if rank and int(rank) < min_rank:  # Only keep ranks < 5000 (top 5000, to ensure more variety)
            results.append({
                "loinc_num": result.get("LOINC_NUM", "Unknown"),
                "long_common_name": result.get("LONG_COMMON_NAME", "Unknown"),
                "component": result.get("COMPONENT", "Unknown"),
                "system": result.get("SYSTEM", "Unknown"),
                "property": result.get("PROPERTY", "Unknown"),
                "rank": int(rank)  # Convert to integer
            })

    # take randomly num_results amount of documents that are in top 5000
    df_results = pd.DataFrame(results).sample(n=min(num_results, len(results)), random_state=42)
    return df_results

In [ ]:
#we save this dataset, so we do not have to run it that many times
top5k_entries = get_top_loinc_entries()
len(top5k_entries)

5000

In [44]:
# add terms to sheets
def add_terms_to_sheets(file_path, sheet_name = None, related = 0, unrelated = 0, top_loinc_entries = top5k_entries):
    """
    Reads the Excel file and adds x related and y unrelated documents to each sheet.
    """
    # Read all sheets from the Excel file
    excel_sheets = pd.read_excel(file_path, sheet_name)

    # Dictionary to store updated data
    updated_sheets = {}

    # Iterate through each sheet in the Excel file
    for sheet_name, df in excel_sheets.items():
        print(f"Adding documents to sheet: {sheet_name}")

        # Get x_related relevant documents based on the sheet name (query)
        df_related = get_allrelated_loincs(sheet_name) #all related
        df_x_related = df_related.sample(n=min(related, len(df_related)), random_state=42)  #randomly xnumber of related of the total

        # Get y_unrelated documents ensuring no duplicates
        existing_loincs = set(df_related["loinc_num"].astype(str))  # all related loincs
        df_unrelated = top_loinc_entries[~top_loinc_entries["loinc_num"].isin(existing_loincs)]
        df_y_unrelated = df_unrelated.sample(n=min(unrelated, len(df_unrelated)), random_state=42)  #randomly ynumber of unrelated of the total
        df_y_unrelated["inSearch"] = 0 #add the column inSearch 

        # Combine original data with new documents
        df_updated = pd.concat([df, df_x_related, df_y_unrelated], ignore_index=True)

        # Save updated sheet data
        updated_sheets[sheet_name] = df_updated

    # Save the updated Excel file with all new rows
    with pd.ExcelWriter(file_path, engine="openpyxl", mode="w") as writer:
        for sheet_name, df in updated_sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    print("Related and unrelated documents have been successfully added.")

In [ ]:
# add terms to the defect sheets
add_terms_to_sheets("loinc_ranks_query.xlsx", None, 50, 400, top5k_entries)

Adding documents to sheet: glucose in blood
Adding documents to sheet: bilirubin in plasma
Adding documents to sheet: White blood cells count
Related and unrelated documents have been successfully added.


## Adding new queries

In [45]:
def add_new_query_sheet(new_query, file_path="loinc_ranks_query_terms.xlsx", related = 50, unrelated = 500, top_loinc_entries = top5k_entries):
    """
    Adds a completely new query and its related documents as a new sheet in the Excel file.
    """
    # Read all existing sheets
    excel_sheets = pd.read_excel(file_path, sheet_name=None)

    print(f"Adding new query: {new_query}")

    # Get x_related relevant documents based on the sheet name (query)
    df_related = get_allrelated_loincs(new_query) #all related
    df_x_related = df_related.sample(n=min(related, len(df_related)), random_state=42)  #randomly xnumber of related of the total

    # Get y_unrelated documents ensuring no duplicates
    existing_loincs = set(df_related["loinc_num"].astype(str))  # all related loincs
    df_unrelated = top_loinc_entries[~top_loinc_entries["loinc_num"].isin(existing_loincs)]
    df_y_unrelated = df_unrelated.sample(n=min(unrelated, len(df_unrelated)), random_state=42)  #randomly ynumber of unrelated of the total
    df_y_unrelated["inSearch"] = 0 #add the column inSearch 

    # Combine original data with new documents
    df_new_sheet = pd.concat([df_x_related, df_y_unrelated], ignore_index=True)
    

    # Add the new query as a sheet in the Excel file
    with pd.ExcelWriter(file_path, engine="openpyxl", mode="w") as writer:
        for sheet_name, df in excel_sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # Add the new sheet
        df_new_sheet.to_excel(writer, sheet_name=new_query, index=False)

    print(f"New sheet '{new_query}' added successfully.")

In [46]:
add_new_query_sheet("cholesterol in blood")

Adding new query: cholesterol in blood
New sheet 'cholesterol in blood' added successfully.


In [47]:
add_new_query_sheet("fever virus")

Adding new query: fever virus
New sheet 'fever virus' added successfully.


In [48]:
add_new_query_sheet("calcium oxalate crystals")

Adding new query: calcium oxalate crystals
New sheet 'calcium oxalate crystals' added successfully.


In [49]:
add_new_query_sheet("iron")

Adding new query: iron
New sheet 'iron' added successfully.


In [50]:
add_new_query_sheet("PrThr")

Adding new query: PrThr
New sheet 'PrThr' added successfully.


## Creating testing set

In [53]:
def add_top_loinc_sheet(file_path, number = 500, dataset = top5k_entries):
    """
    Adds a new sheet with the top 200 LOINC entries where rank > 3000 to the existing Excel file.
    """
    # Read all existing sheets
    excel_sheets = pd.read_excel(file_path, sheet_name=None)

    # Fetch top-ranked LOINC entries with rank > 3000
    #df_top_loinc = get_top_loinc_entries(min_rank=3000, num_results=200)
    df_top_loinc = dataset.sample(n=min(number, len(dataset)), random_state=42)
    #df_top_loinc = df_top_loinc.drop(columns=["inSearch"])

    # Save updated Excel file with the new sheet
    with pd.ExcelWriter(file_path, engine="openpyxl", mode="w") as writer:
        for sheet_name, df in excel_sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # Add the new sheet with top-ranked LOINC entries
        df_top_loinc.to_excel(writer, sheet_name="testing", index=False)

    print("New sheet 'testing' added successfully.")

In [54]:
add_top_loinc_sheet("loinc_ranks_query_terms.xlsx")

New sheet 'testing' added successfully.
